In [108]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve

In [69]:
loaded_model = xgb.Booster()
loaded_model.load_model('LeoPayu_PG_60_in_15mob_Vin.json')
print("Model Loaded Successfully")

Model Loaded Successfully


In [70]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)


********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [71]:
# Drop Null values of Target 
df = model_data.dropna(subset=['90_in_24mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [72]:
loaded_model.feature_names

['distinct_merchant',
 'SUM(raw.amount WHERE action = 1)/SUM(raw.amount)#ever',
 'MEAN(raw.amount WHERE device_os_category = Windows)#ever',
 'NUM_UNIQUE(pg_success_raw_data.month_end_start_category)',
 'max_one_time_spent',
 'amt_first_android_txn',
 'MAX(pg_success_raw_data.amount)',
 'top_payment_mode_count',
 'amt_first_cc_txn',
 'COUNT(raw WHERE amount_category = 6)/COUNT(raw)#ever',
 'ENTROPY(raw.mode_cleaned)#720',
 'COUNT(pg_success_raw_data WHERE revised_merchant_category = bill_pay)',
 'YEAR(first_pg_failed_raw_data_time)']

In [73]:
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]

In [74]:
features_list=['distinct_merchant',
 'SUM(raw.amount WHERE action = 1)/SUM(raw.amount)#ever',
 'MEAN(raw.amount WHERE device_os_category = Windows)#ever',
 'NUM_UNIQUE(pg_success_raw_data.month_end_start_category)',
 'max_one_time_spent',
 'amt_first_android_txn',
 'MAX(pg_success_raw_data.amount)',
 'top_payment_mode_count',
 'amt_first_cc_txn',
 'COUNT(raw WHERE amount_category = 6)/COUNT(raw)#ever',
 'ENTROPY(raw.mode_cleaned)#720',
 'COUNT(pg_success_raw_data WHERE revised_merchant_category = bill_pay)',
 'YEAR(first_pg_failed_raw_data_time)',
 'month_year',
 '90_in_24mob']

In [75]:
# Selected Features for model build purpose
model_df = model_df1[features_list]

In [76]:
oot_df = oot_df1[features_list]

In [77]:
model_df.shape, oot_df.shape

((39010, 15), (12128, 15))

In [78]:
model_df['90_in_24mob'].value_counts(dropna=False)

90_in_24mob
0.0    32389
1.0     6621
Name: count, dtype: int64

In [79]:
# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '90_in_24mob']) # Adjust 'target' to your actual target column name
y_model = model_df['90_in_24mob']

In [80]:
X_oot = oot_df.drop(columns=['month_year', '90_in_24mob'])
y_oot = oot_df['90_in_24mob']

In [81]:
#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3225027/2233446063.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)


In [82]:
for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3225027/1979277635.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_oot[i].fillna(-9999,inplace=True)


In [83]:
X_oot.shape,y_oot.shape

((12128, 13), (12128,))

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [85]:
X_train.shape, X_test.shape

((23406, 13), (15604, 13))

In [86]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [87]:
def decile_analysis(df, prediction_col, actual_col,label_good= 0, label_bad= 1 ):
    
    df['Decile'] = pd.qcut(df[prediction_col],10, duplicates='drop', labels = False)+1
    # count_good = df.groupby("Decile")[actual_col]==0.shape[0]
    # count_bad = df.groupby("Decile")[actual_col]==1.shape[0]
   
    decile_summary = df.groupby('Decile').agg(
        mean_actual = (actual_col, 'mean'),
        mean_prediction =( prediction_col, 'mean'),
        max_prediction =( prediction_col, 'max'),
        min_prediction =( prediction_col, 'min'),
        
        count=(actual_col, 'count'),
        label_good=(actual_col,lambda x: (x == label_good).sum()),
        label_bad=(actual_col,lambda x: (x == label_bad).sum())
    ).sort_index(ascending =False)
        
        # label_good =(actual_col == label_good).sum(),
        # label_bad =(actual_col == label_good).sum()).sort_index(ascending =False)
        
    auc = roc_auc_score(df[actual_col], df[prediction_col])
    gini = 2*auc-1
    fpr,tpr,thresholds = roc_curve(df[actual_col], df[prediction_col])
    ks = (tpr-fpr).max()
    return decile_summary, auc, gini, ks
    #print(f'Gini:{gini}, AUC:{auc}, KS:{ks}')

In [46]:
# params = {
#     'objective': 'binary:logistic',
#     'eval_metric': 'auc',
#     'learning_rate': 0.01,
#      'max_depth': 2,

#     'subsample': 0.3,
#     'colsample_bytree': 0.3,
#     # 'min_child_weight':25,

#     #'lambda': 2.6,
#     'lambda': 7.5,
#     'alpha':5.5,
#     'gamma':5.0,

#     # 'scale_pos_weight':4.0,
#     'random_state': 24
    
# }

# evals = [(dtrain, 'train'), (dtest, 'eval')]
# xgb_model = xgb.train(
#     params,
#     dtrain,
#     num_boost_round=500,
#     early_stopping_rounds=50,
#     evals=evals,
#     verbose_eval=10
# )

In [88]:
# 4. **Predictions**
X_train['y_actual'] = y_train
X_train['60_15_y_pred'] = loaded_model.predict(dtrain)
X_train['LID'] = model_df1.loc[X_train.index,'latest_feature_id'].values

X_test['y_actual'] = y_test
X_test['60_15_y_pred'] = loaded_model.predict(dtest)
X_test['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot['y_actual'] = y_oot
X_oot['60_15_y_pred'] = loaded_model.predict(doot)
X_oot['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

In [89]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train, '60_15_y_pred', 'y_actual')
test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test, '60_15_y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot, '60_15_y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.58, Gini: 0.16, KS: 0.12
Test AUC: 0.56, Gini: 0.12, KS: 0.10
OOT AUC: 0.57, Gini: 0.14, KS: 0.11


In [90]:
Train = X_train[['LID','y_actual','60_15_y_pred']]
Test = X_test[['LID','y_actual','60_15_y_pred']]
Oot = X_oot[['LID','y_actual','60_15_y_pred']]


In [91]:
Train.to_csv('PG_Train_Pred_60_15_Vin.csv',index=False)
Test.to_csv('PG_Test_Pred_60_15_Vin.csv',index=False)
Oot.to_csv('PG_Oot_Pred_60_15_Vin.csv',index=False)


In [92]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
9          0.244868         0.150563        0.160418        0.147942    682   
8          0.202639         0.147714        0.147923        0.145147   2956   
7          0.198681         0.140055        0.145146        0.136867   1213   
6          0.178895         0.135752        0.136862        0.134798   1213   
5          0.164880         0.134097        0.134797        0.133413   1213   
4          0.167492         0.132680        0.133413        0.131777   1212   
3          0.139324         0.130581        0.131775        0.129371   1213   
2          0.138500         0.128566        0.129370        0.127835   1213   
1          0.103050         0.127024        0.127834        0.125089   1213   

        label_good  label_bad  
Decile                         
9              515        167  
8             2357        599  
7              972        241  
6              996        217  
5             1013        200  
4             1009        203  
3             1044        169  
2             1045        168  
1             1088        125

In [93]:
loaded_model = xgb.Booster()
loaded_model.load_model('LeoPayu_pg_15_in_24mob_Vin.json')
print("Model Loaded Successfully")

Model Loaded Successfully


In [94]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded *********')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)


********* Data loaded *********
(478328, 2831)
(478328, 2831)


In [95]:
# Drop Null values of Target 
df = model_data.dropna(subset=['90_in_24mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [96]:
loaded_model.feature_names

['TIME_SINCE_FIRST(raw.created_timestamp WHERE hour_part2 = 1)#ever',
 'NUM_UNIQUE(pg_success_raw_data.YEAR(created_time_ist))',
 'days_since_first_premium_bank_trans',
 'NUM_UNIQUE(pg_success_raw_data.bank_title)',
 'MEAN(raw.amount WHERE mode_cleaned = 2)#ever',
 'max_one_time_spent',
 'AVG_TIME_BETWEEN(raw.created_timestamp WHERE mode_cleaned = 2)#ever',
 'distinct_merchant',
 'days_since_first_ecom_trans',
 'days_on_pg']

In [97]:
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]

In [98]:
features_list=['TIME_SINCE_FIRST(raw.created_timestamp WHERE hour_part2 = 1)#ever',
 'NUM_UNIQUE(pg_success_raw_data.YEAR(created_time_ist))',
 'days_since_first_premium_bank_trans',
 'NUM_UNIQUE(pg_success_raw_data.bank_title)',
 'MEAN(raw.amount WHERE mode_cleaned = 2)#ever',
 'max_one_time_spent',
 'AVG_TIME_BETWEEN(raw.created_timestamp WHERE mode_cleaned = 2)#ever',
 'distinct_merchant',
 'days_since_first_ecom_trans',
 'days_on_pg',
    'month_year',
'90_in_24mob']

In [99]:
# Selected Features for model build purpose
model_df = model_df1[features_list]
oot_df = oot_df1[features_list]

# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '90_in_24mob']) # Adjust 'target' to your actual target column name
y_model = model_df['90_in_24mob']

X_oot = oot_df.drop(columns=['month_year', '90_in_24mob'])
y_oot = oot_df['90_in_24mob']

#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)
    
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


# 4. **Predictions**
X_train['y_actual'] = y_train
X_train['15_24_y_pred'] = loaded_model.predict(dtrain)
X_train['LID'] = model_df1.loc[X_train.index,'latest_feature_id'].values

X_test['y_actual'] = y_test
X_test['15_24_y_pred'] = loaded_model.predict(dtest)
X_test['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot['y_actual'] = y_oot
X_oot['15_24_y_pred'] = loaded_model.predict(doot)
X_oot['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values


# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train, '15_24_y_pred', 'y_actual')
test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test, '15_24_y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot, '15_24_y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.58, Gini: 0.16, KS: 0.13
Test AUC: 0.56, Gini: 0.11, KS: 0.09
OOT AUC: 0.56, Gini: 0.12, KS: 0.10


/tmp/ipykernel_3225027/736859902.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3225027/736859902.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

In [100]:
Train = X_train[['LID','y_actual','15_24_y_pred']]
Test = X_test[['LID','y_actual','15_24_y_pred']]
Oot = X_oot[['LID','y_actual','15_24_y_pred']]


In [101]:
Train.to_csv('Pg_Train_Pred_G15_B24_Vin.csv',index=False)
Test.to_csv('Pg_Test_Pred_G15_B24_Vin.csv',index=False)
Oot.to_csv('Pg_Oot_Pred_G15_B24_Vin.csv',index=False)


In [102]:
# 60-15
b1_train = pd.read_csv('PG_Train_Pred_60_15_Vin.csv')
b1_test = pd.read_csv('PG_Test_Pred_60_15_Vin.csv')
b1_oot = pd.read_csv('PG_Oot_Pred_60_15_Vin.csv')

#15-24
b1_train_1 = pd.read_csv('Pg_Train_Pred_G15_B24_Vin.csv')
b1_test_1 = pd.read_csv('Pg_Test_Pred_G15_B24_Vin.csv')
b1_oot_1 = pd.read_csv('Pg_Oot_Pred_G15_B24_Vin.csv')

In [103]:
b1_train.drop(columns=['y_actual'],inplace=True)
b1_test.drop(columns=['y_actual'],inplace=True)
b1_oot.drop(columns=['y_actual'],inplace=True)


In [104]:
Train = pd.merge(b1_train,b1_train_1, how='left',on='LID')

In [105]:
Test= pd.merge(b1_test,b1_test_1, how='left',on='LID')


In [106]:
Oot= pd.merge(b1_oot,b1_oot_1, how='left',on='LID')


In [107]:
Train.to_csv('PG_Train_Data_ensemble_Vin.csv',index=False)
Test.to_csv('PG_Test_Data_ensemble_Vin.csv',index=False)
Oot.to_csv('PG_Oot_Data_ensemble_Vin.csv',index=False)
